```
PLEASE NOTE THAT THE BELOW HAS NOT BE THOROUGHLY CLEANED, IT REFLECTS THE CALCULATIONS MADE TO INFORM THE MAIN MODEL FUNCTION
```

# Prepare various datasets

In [ ]:
library(tidyverse)
library(lubridate)


### ECDC data worldwide

Script from Imperial report #13

In [ ]:
url <- "https://opendata.ecdc.europa.eu/covid19/casedistribution/csv"

url_page <- "https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide"
tryCatch({
  code <- download.file(url, "data/COVID-19-up-to-date.csv")
  if (code != 0) {
    stop("Error downloading file")
  }
},
error = function(e) {
  stop(sprintf("Error downloading file '%s': %s, please check %s",
               url, e$message, url_page))
})


d <- read.csv("data/COVID-19-up-to-date.csv", stringsAsFactors = FALSE)
d$t <- lubridate::decimal_date(as.Date(d$dateRep, format = "%d/%m/%Y"))
d <- d[order(d$'countriesAndTerritories', d$t, decreasing = FALSE), ]
names(d)[names(d) == "countriesAndTerritories"] <- "Countries.and.territories"
names(d)[names(d) == "deaths"] <- "Deaths"
names(d)[names(d) == "cases"] <- "Cases"
names(d)[names(d) == "dateRep"] <- "DateRep"
saveRDS(d, "data/COVID-19-up-to-date.rds")

### SOCRATES - Social distancing beta

Generated manually using https://lwillem.shinyapps.io/socrates_rshiny/ "Distancing" option,
with settings:

UK Mossong 2008
- Schools closed
- Telework 60% observed, 70% target
- Social distancing 80% transport, 90% leisure, 90% otherplace

Double click to reveal data
<!--
$matrix
         contact.age.group
             [0,10)   [10,20)   [20,30)   [30,40)   [40,50)   [50,60)   [60,70)
  [0,10)  4.4524648 1.0023677 0.9267597 1.8494522 0.8416188 0.4184347 0.3653064
  [10,20) 0.9028817 6.2891541 0.7661398 1.1677677 1.5084552 0.5513150 0.3290871
  [20,30) 0.8484163 0.7786568 1.9985976 1.0581466 1.1608058 0.7290598 0.3352099
  [30,40) 1.4630051 1.0255466 0.9143379 1.7649299 1.1682960 0.6524238 0.5153032
  [40,50) 0.6827956 1.3586386 1.0287102 1.1981894 1.7330835 0.7400668 0.4800984
  [50,60) 0.3903293 0.5709519 0.7428905 0.7693618 0.8509404 1.0113230 0.4037793
  [60,70) 0.4392010 0.4392513 0.4402314 0.7831881 0.7114772 0.5204110 0.7916867
  [70,79) 0.2097901 0.4888515 0.1921582 0.2887334 0.6030292 0.3634602 0.3458098
  79+     0.1201383 0.3895300 0.1902486 0.2879470 1.5277207 2.1654861 0.3425987
         contact.age.group
             [70,79)         79+
  [0,10)  0.11661007 0.006675041
  [10,20) 0.24475509 0.019494732
  [20,30) 0.09778038 0.009676893
  [30,40) 0.12695528 0.012655749
  [40,50) 0.27193473 0.068863943
  [50,60) 0.18845664 0.112235834
  [60,70) 0.23109700 0.022885705
  [70,79) 0.60200135 0.020608840
  79+     0.20617308 1.018981751

$mij_ratio
         contact.age.group
             [0,10)   [10,20)   [20,30)   [30,40)   [40,50)   [50,60)   [60,70)
  [0,10)  0.8724908 0.7786869 0.7110732 0.7989095 0.7499990 0.6901133 0.8059413
  [10,20) 0.7786869 0.7860275 0.6130213 0.7718294 0.7986402 0.6596220 0.7628860
  [20,30) 0.7110732 0.6130213 0.5626967 0.5836886 0.6428855 0.5646897 0.4730505
  [30,40) 0.7989095 0.7718294 0.5836886 0.6406482 0.6003155 0.5283291 0.5969840
  [40,50) 0.7499990 0.7986402 0.6428855 0.6003155 0.5957767 0.5375141 0.5585658
  [50,60) 0.6901133 0.6596220 0.5646897 0.5283291 0.5375141 0.5244481 0.4190457
  [60,70) 0.8059413 0.7628860 0.4730505 0.5969840 0.5585658 0.4190457 0.5960359
  [70,79) 0.6561738 0.7133536 0.3933889 0.5189821 0.5561491 0.4069975 0.3489019
  79+     0.5974460 0.6008924 0.4494123 0.3209152 0.6231519 0.6116938 0.2416726
         contact.age.group
            [70,79)       79+
  [0,10)  0.6561738 0.5974460
  [10,20) 0.7133536 0.6008924
  [20,30) 0.3933889 0.4494123
  [30,40) 0.5189821 0.3209152
  [40,50) 0.5561491 0.6231519
  [50,60) 0.4069975 0.6116938
  [60,70) 0.3489019 0.2416726
  [70,79) 0.5116825 0.1589608
  79+     0.1589608 1.0000000

$notes
     [,1]                                              
[1,] "ratio = with intervention / without intervention"
[2,] "All schools are closed"                          
[3,] "Increased telework (70% instead of 60%)"         
[4,] "Social distancing for Transport: 80% reduction"  
[5,] "Social distancing for Leisure: 90% reduction"    
[6,] "Social distancing for Otherplace: 90% reduction" 
[7,] "Social distancing for Work: 25% reduction"
!-->

### Hospital episode statistics

From UK NHS Digital 2015-16 report we can get age distribution (see saved png in data/)
source: https://webarchive.nationalarchives.gov.uk/20180328130140/http://digital.nhs.uk/catalogue/PUB22378

The 0-4 age includes births therefore are ignored, and the 0-9 range is extrapolated from age 5-9

In [ ]:
finishedAdmissionEpisodes = list(
    '0-9'=306259*2,
    '10-19'=250177+393755,
    '20-29'=673309+878408,
    '30-39'=903372+743633,
    '40-49'=701205+834100,
    '50-59'=959687+1037374,
    '60-69'=1065839+1336644,
    '70-79'=1270295+1239246,
    '80+' = 1033567+713075+423942
)

currentPopulation = list(
    '0-9'=3357463*2,
    '10-19'=3000295+3213289,
    '20-29'=3592257+3757968,
    '30-39'=3728028+3470918,
    '40-49'=3636454+3889369,
    '50-59'=3811000+3278322,
    '60-69'=2904721+3017135,
    '70-79'=2268620+1804228,
    '80+' = 1326300+820321+474968
)

df_hospitalisation = data.frame(
    finishedAdmissionEpisodes=unlist(finishedAdmissionEpisodes), 
    currentPopulation=unlist(currentPopulation)
)

df_hospitalisation['ageRelativeHospitalisationDailyRates'] = df_hospitalisation$finishedAdmissionEpisodes/(365*df_hospitalisation$currentPopulation)

# The Scottish ISD dataset estimate for overall hospitalisation daily rate is 0.0005292510254722236
# Here it is 0.000754683533340698, similar order of magnitude (good to double check!)
# sum(df_hospitalisation$relativeHospitalisationDailyRates*df_hospitalisation$currentPopulation/sum(df_hospitalisation$currentPopulation))

# Save dataframe as csv to load in python model
write.csv(df_hospitalisation, "data/clean_hosp-epis-stat-admi-summ-rep-2015-16-rep_table_6.csv")

In [ ]:
df_hospitalisation

In [ ]:
# More detailed dataset that includes length of stay
# From NHS digital: https://digital.nhs.uk/data-and-information/find-data-and-publications/supplementary-information/2019-supplementary-information-files/mean-and-median-length-of-stay-of-hospital-inpatient-episodes

# Mean and median length of stay of hospital inpatient episodes
# Published: 15 April 2019
# Time Frame: 2017/18
# Summary: Counts of ordinary episodes and the mean and median length of stay (in days), by Hospital Provider, by 10 year age bands, for the year 2017-18

df_hosp_xlsx = xlsx::read.xlsx("data/10641_LoS_age_provider_suppressed.xlsx", 2, rowIndex=5:431, colIndex=4:37)

df_hosp_xlsx[df_hosp_xlsx=="*"] = 0
df_hosp_xlsx[df_hosp_xlsx==NA] = 0


# Age-grouped mean length of stay (number of episodes * average length of stay, averaged over NHS trusts)
totalStaysByAge = colSums(as.matrix(sapply(df_hosp_xlsx[,1:10], function(x)as.numeric(as.character(x)))))

ageHospitalMeanLengthOfStay = colSums(
    as.matrix(sapply(df_hosp_xlsx[,1:10], function(x)as.numeric(as.character(x)))) *
    as.matrix(sapply(df_hosp_xlsx[,12:21], function(x)as.numeric(as.character(x))))
) / totalStaysByAge

     
# Merge age 80-89 and 90+ into 80+ (as the rest of our data)
ageHospitalMeanLengthOfStay[9] = (
    ageHospitalMeanLengthOfStay[9] * totalStaysByAge[9] + 
    ageHospitalMeanLengthOfStay[10] * totalStaysByAge[10]
) / (totalStaysByAge[9] + totalStaysByAge[10])
ageHospitalMeanLengthOfStay = ageHospitalMeanLengthOfStay[-10]
names(ageHospitalMeanLengthOfStay)[9] = "X80+"

df_lengthofstay = data.frame(ageHospitalMeanLengthOfStay=ageHospitalMeanLengthOfStay)         
write.csv(df_lengthofstay, "data/clean_10641_LoS_age_provider_suppressed.csv")
              

In [ ]:
df_lengthofstay

### NHS workforce statistics

Latest from:
https://digital.nhs.uk/data-and-information/publications/statistical/nhs-workforce-statistics/december-2019

Note that this is England, and thus ratio of hospital staff should be normalised to England-only age distribution

Age distribution does not seem available, so the main information we need here is "667,965 Professionally qualified clinical staff". I'll assume for simplicity that their age distribution reflects the general population age distribution between age 20-69, this can be clarified with more data in needed.

The population figures in "df_hospitalisation" also reflect England, we can use that as a baseline

In [ ]:
tmp = df_hospitalisation$currentPopulation
tmp[1:2] = 0
tmp[8:9] = 0
df_nhsclinicalstaff = data.frame(
    ageNhsClinicalStaffPopulationRatio = (667965*tmp/sum(tmp))/df_hospitalisation$currentPopulation
)

write.csv(df_nhsclinicalstaff, "data/clean_nhsclinicalstaff.csv")